In [11]:
import os 
import math
import random
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from numpy import count_nonzero

dir_ = '../data/'
group_dir_ = '../data/groups/random'

In [12]:
file_name = 'normalized_minmax_filter_track_5_user_100.csv'
svd = pd.read_pickle(os.path.join(dir_, 'prediction_svd_top_N_' + file_name[:-3] + 'pkl'))
test = pd.read_pickle(os.path.join(dir_, 'test_' + file_name[:-3] + 'pkl'))
test.sort_values(by=['uid','tid'])

file_name = 'normalized_popularity_filter_track_5_user_100.pkl'
pop_count = pd.read_pickle(os.path.join(dir_, file_name))

num_user = len(svd['uid'].unique())
num_user

220

In [13]:
groups_n =[]
group_sizes = ['2', '3', '4', '5']
for i in range(len(group_sizes)):
    groups = np.loadtxt(os.path.join(group_dir_, group_sizes[i] + '.csv'), delimiter=',')
    groups_n.append(groups)

In [14]:
for i in groups_n:
    print(len(i[0]))

2
3
4
5


In [15]:
svd['count'] = 1
svd[:5]

,uid,tid,rating,count
0,0,3,2.544071,1
1,0,7,2.545822,1
2,0,18,2.152490,1
3,0,20,1.882396,1
4,0,31,2.410806,1


In [16]:
pop_count = pop_count.sort_values(by=['count'],  ascending=False)
pop_count[:10]

,tid,count,rating
166,166,3666,573.25
457,457,2042,162.00
6338,6338,1896,153.75
80,80,1835,412.25
1364,1364,1792,327.50
350,350,1667,359.75
7209,7209,1454,400.50
6282,6282,1397,17.75
13104,13104,1368,57.50
3761,3761,1340,354.50


In [17]:
# Linear
# size = int(len(pop_rating) // 5)
# pop_normalized = pd.DataFrame()
# for i in range(5):
#     if i == 0:
#         pop = pop_count[:size].copy()
#     elif i == 4:
#         pop = pop_count[size*4:].copy()
#     else:
#         pop = pop_count[size*i:size*(i+1)].copy()
#     pop['rating'] = (5 - i)
#     pop_normalized = pop_normalized.append(pop)
# pop_normalized = pop_normalized[['tid', 'rating']]

In [18]:
# min_max
max_rating = pop_count.iloc[0]['count']
pop_normalized = pop_count.copy()
pop_normalized = pop_normalized[['tid', 'count']]
pop_normalized['rating'] = pop_normalized['count']
pop_normalized['rating'] /= max_rating
pop_normalized['rating'] *= 5
pop_normalized[:10]

,tid,count,rating
166,166,3666,5.000000
457,457,2042,2.785052
6338,6338,1896,2.585925
80,80,1835,2.502728
1364,1364,1792,2.444081
350,350,1667,2.273595
7209,7209,1454,1.983088
6282,6282,1397,1.905346
13104,13104,1368,1.865794
3761,3761,1340,1.827605


In [19]:
lambdas = []
for i in range(11):
    lambdas.append(i/10)
lambdas

[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

In [20]:
for lambda_ in tqdm(lambdas):
    top_n_size = 4
    top_n_lists_g = []
    for groups in groups_n:

        group_size = len(groups[0])
        top_n_lists = []

        for group in groups:
            rating_table = pd.DataFrame() 
            for member in group:
                prediction = svd[svd['uid'] == member].copy()
                if len(rating_table) == 0:
                    rating_table = prediction
                else:
                    rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
            rating_table['rating'] /= group_size 
            rating_table['rating'] *= lambda_
            pop = pop_normalized.copy()
            pop['rating'] *= (1 - lambda_)
            rating_table = rating_table.set_index('tid').add(pop.set_index('tid'), fill_value=0).reset_index()
    #         rating_table = rating_table[rating_table['count'] == group_size]
            rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
            rating_table = rating_table[:top_n_size]
            top_n_list = []
            for _, row in rating_table.iterrows():
                top_n_list.append(row[0])
            top_n_lists.append(top_n_list)
        top_n_lists_g.append(top_n_lists)   
        
#     print('lambda = ' + str(lambda_))
    threshold = 0.0
    avg_precisions = []
    for i in range(len(groups_n)):
        groups = groups_n[i]
        top_n_lists = top_n_lists_g[i]

        precisions = []
        for j in (range(len(groups))):

            group = groups[j]
            top_n_list = top_n_lists[j]

            high_rating = 0

            for k in range(top_n_size):
                for l in range(len(group)):
                    uid = group[l]
                    tid = top_n_list[k]
                    t = test[test['uid'] == uid]
                    t = t[t['tid'] == tid]
                    if len(t) > 0 and t.iloc[0]['rating'] > threshold:
                        high_rating += 1
                        break

            precision = high_rating / top_n_size
            precisions.append(precision)

        avg_precision = 0
        for precision in precisions:
            avg_precision += precision
        avg_precision /= len(precisions)
        avg_precisions.append(avg_precision)
    print(avg_precisions)
#     print(sum(avg_precisions) / len(avg_precisions))
#     print('----------------------------------------') 

[0.1375, 0.1875, 0.26666666666666666, 0.3125]
[0.18333333333333332, 0.225, 0.30833333333333335, 0.375]
[0.2, 0.23125, 0.31666666666666665, 0.3958333333333333]
[0.24166666666666667, 0.25625, 0.325, 0.40625]
[0.2833333333333333, 0.29375, 0.325, 0.4583333333333333]
[0.3541666666666667, 0.3625, 0.35833333333333334, 0.4791666666666667]
[0.4041666666666667, 0.41875, 0.4083333333333333, 0.5208333333333334]
[0.4166666666666667, 0.45, 0.525, 0.6041666666666666]
[0.4083333333333333, 0.48125, 0.5333333333333333, 0.6458333333333334]
[0.4083333333333333, 0.4625, 0.5416666666666666, 0.65625]
[0.3333333333333333, 0.39375, 0.5, 0.5625]



In [195]:
alpha = 0.5
threshold = 0.0
for i in tqdm(range(len(groups_n))):
    
    groups = groups_n[i]
    top_n_lists = top_n_lists_g[i]
    
    satisfactions = []
    for j in (range(len(groups))):
        
        group = groups[j]
        top_n_list = top_n_lists[j]
        
        high_ratings_t = np.zeros(top_n_size)
        high_ratings_u = np.zeros(len(group))
        
        for k in range(top_n_size):
            for l in range(len(group)):
                uid = group[l]
                tid = top_n_list[k]
                t = test[test['uid'] == uid]
                t = t[t['tid'] == tid]
                if len(t) > 0 and t.iloc[0]['rating'] > threshold:
                    high_ratings_t[k] += 1
                    high_ratings_u[l] += 1
        
#         satisfaction_t = 0
#         for k in range(top_n_size):
#             satisfaction_t += math.log(high_ratings_t[k]+1, 10) / math.log(len(group)+1, 10)
#         satisfaction_t /= top_n_size
        
#         satisfaction_u = 0
#         for k in range(len(group)):
#             satisfaction_u += math.log(high_ratings_u[k]+1, 10) / math.log(top_n_size+1, 10)
#         satisfaction_u /= len(group)
        satisfaction_t = math.log(max(high_ratings_t)+1, 10) / math.log(len(group)+1, 10)
        satisfaction_u = math.log(min(high_ratings_u)+1, 10) / math.log(top_n_size+1, 10)
        
        satisfaction = alpha*satisfaction_t + (1-alpha)*satisfaction_u
        satisfactions.append(satisfaction)
    
    avg_satisfaction = 0
    for satisfaction in satisfactions:
        avg_satisfaction += satisfaction
    avg_satisfaction /= len(satisfactions)
    print(avg_satisfaction)
        

0.4384689835799772
0.34093950544457624
0.3358646172870294
0.33769451038646087

